# Delete Printer

This module delete a printer.

Module interaction:

    Environement selection
    Premise selection
    Printer selection in the premise


## Select an environment

In [ ]:
%load_ext autoreload
%autoreload 2
from libConfUtils import chooseEnv as chooseEnv
from libConfUtils import deviceUtils as deviceUtils
from libConfUtils import povUtils as povUtils
from libConfUtils import uiUtils as uiUtils
from libConfUtils import premiseNhubUtils as premiseNhubUtils
from libConfUtils import settingsUtils as settingsUtil

## Select the Premise where is located the printer.

In [ ]:
#%%
status = True
myEnv = chooseEnv.choosePremise()

In [ ]:
#%% display choice
if not myEnv.get('PremiseCode'):
    text = 'No Premise Found '
    uiUtils.displayMessage('Device search',text)
    print(text)
    status = False
else:  
    print("Organization: " + myEnv.get('urlDP') + " Premise code: "+  myEnv.get('PremiseCode'))

## Load all the printers linked to the premise

In [ ]:
if status:
    listOfDevices = deviceUtils.getPrintersOfPremise(myEnv)

    if not listOfDevices:
        text = "Premise: " + str(myEnv.get('PremiseCode')) + ' No printer found '
        uiUtils.displayMessage('Printer search',text)
        print(text)
        status = False
        
if status:       
    listOfNames = []
    for device in listOfDevices:
        nameOfDevice = device.get('name')
        if not nameOfDevice:
            nameOfDevice = "NoName"
        idOfDevice = device.get('id')
        str2Display =  idOfDevice + ":" +nameOfDevice
        listOfNames.append(str2Display)

### Display the list for selection

In [ ]:
if status:
    deviceC = uiUtils.chooseWithMenu('Devices',listOfNames)
    if not deviceC:
        text = "Premise: " + str(myEnv.get('PremiseCode')) + ' No Printer Selected '
        uiUtils.displayMessage('Devices Selection',text)
        print(text)
        status = False
    else:
        tmp = deviceC.split(':')
        deviceId = tmp[0]
        if not deviceId:
            text = "Premise: " + str(myEnv.get('PremiseCode')) + ' Error in Printer selection: '+deviceC
            uiUtils.displayMessage('Printer Selection',text)
            print(text)
            status = False  

### Request printer definition

In [ ]:
if status:
    deviceDefinition = deviceUtils.getPrinter(myEnv,deviceId)
    if not deviceDefinition:
        text = "Premise: " + str(myEnv.get('PremiseCode')) + ' Error getting printer information: '+ deviceId
        uiUtils.displayMessage('Printer Selection',text)
        print(text)
        status = False  

In [ ]:
if status:
    text = 'Do you want to delete this printer ?' +'\n'
    text  = text + '\n'+ 'Premise: ' + str(myEnv.get('PremiseCode'))
    text  = text +'\n' + 'Printer ID: ' +  deviceId
    text  = text +'\n' + 'Printer IP: ' +  str(deviceDefinition.get('uri'))
    text  = text +'\n' + 'Printer MAC: ' +  str(deviceDefinition.get('mac'))
    text  = text +'\n' + 'Printer NAME: ' +  str(deviceDefinition.get('name'))
    text  = text +'\n' + 'Printer Fixture: ' +  str(deviceDefinition.get('fixtureId'))
    answer = uiUtils.askQuestionYesNo('Printer delete',text)

### Display printer definition

In [ ]:
#%%
if status :
    if answer is None:
        # skip
        print('skip by user')

    elif (answer=='no'):
        #skip
        print('skip by user')

    else : 
        deviceDefinition = deviceUtils.deletePrinter(myEnv,deviceId)
        #uiUtils.displayMessage('Printer suppression','delete the printer '+deviceId)
        text = "Premise: " + str(myEnv.get('PremiseCode'))
        text  = text +'\n' + 'Printer ID: ' +  deviceId + ' deleted'
        uiUtils.displayMessage('Printer deleted',text)